# Interpolation and Extrapolation

This lecture follows closely [Numerical Recipes](https://numerical.recipes/) 2nd Edition in C and 3rd Edition in C++, Chapter 3 "Interpolation and Extrapolation".

## Introduction

In scientific computing and machine learning, interpolation and extrapolation are fundamental tools for estimating function values at new data points based on known information.
* In machine learning, all standard supervised learning tasks can be viewed as interpolation problems in high-dimensional space. Here, models predict outputs **within the range** of their training data.
* However, when attempting to make predictions outside this range, we face significant challenges in making reliable extrapolations.
  Extrapolation is a particularly challenging task because models typically lack information  beyond their training data.

### Interpolation Methods

Interpolation plays a crucial role in scientific computing and machine learning by estimating function values at new data points based on known information.

* Polynomial interpolation is versatile but can exhibit significant oscillations, particularly at the edges of data (Runge's phenomenon).
* This can be mitigated by using rational functions, which offer more stable estimates and are better suited to handle asymptotic behavior.
* Spline interpolation, especially cubic splines, is valued for its smoothness and continuity up to the second derivative. This makes it effective for applications requiring a smooth fit.

### Challenges with Extrapolation

Extrapolation remains a difficult task, yet physics-informed machine learning (PIML) presents a promising avenue.
By embedding known physical laws, such as ordinary differential equations (ODEs), into models, PIML enables extrapolation that aligns with fundamental constraints. This allows for meaningful extensions of predictions beyond the observed data range.

### Distinguishing Interpolation and Function Approximation

Interpolation and function approximation are related but distinct tasks:

* Interpolation estimates values at specified points within a given dataset.
* In contrast, function approximation creates a simplified function to replace a more complex one.
  This approach can be used to sample additional points as needed.
* (See [Numerical Recipes](https://numerical.recipes/) Chapter 5 for function approximation.)

### Limitations of Interpolation

Even the most sophisticated interpolation schemes can fail when faced with pathological functions.
For instance, consider a function that behaves smoothly except for a slight singularity at a certain point:
\begin{align}
f(x) = 3x^2 + \frac{1}{\pi^4}\ln\left[(\pi - x)^2\right] + 1
\end{align}

Interpolation based on values close to but not precisely at that singularity will likely produce an inaccurate result.

In [ ]:
import numpy as np

def f(x):
    return 3 * x**2 + np.log((np.pi - x)**2) / np.pi**4 + 1

x1 = np.linspace(3.13, 3.16, 3+1)
x2 = np.linspace(3.13, 3.16, 30+1)
x3 = np.linspace(3.13, 3.16, 300+1)
x4 = np.linspace(3.13, 3.16, 3000+1)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(x4, f(x4),       label='3001 points')
plt.plot(x3, f(x3), '--', label='301 points')
plt.plot(x2, f(x2), 'o:', label='31 points')
plt.plot(x1, f(x1), 'o-', label='4 points')
plt.legend()

These cases highlight the importance of having some error estimates in interpolation routines.

## Preliminaries: Searching an Ordered Table

In many interpolation tasks, especially with irregularly sampled data, the process begins with a critical first step: identifying the nearest points surrounding the target interpolation value.

Unlike regularly spaced data on a uniform grid, where adjacent points are easy to locate by simple indexing, randomly sampled or unevenly spaced data requires additional steps to find nearby values.
This searching step can be as computationally intensive as the interpolation itself, so efficient search methods are essential to maintain overall performance.

In Numerical Recipes, two primary methods are presented for this purpose: bisection and hunting.
Each is suited to different scenarios, depending on whether interpolation points tend to be close to one another or scattered randomly.

### Linear Search

As a reference, we will implement a linear search:

In [ ]:
def linear(xs, target):
    for l in range(len(xs)): # purposely use for-loop to avoid C optimization in numpy
        if xs[l] >= target:
            return l-1

In [ ]:
import numpy as np

for _ in range(10):
    xs = np.sort(np.random.uniform(0, 100, 10))
    v  = np.random.uniform(min(xs), max(xs))
    i  = linear(xs, v)
    print(f'{xs[i]} <= {v} < {xs[i+1]}')

### Bisection Search

Bisection search is a reliable method that works by dividing the search interval in half with each step until the target value's position is found.
Given a sorted array of $N$ data points, this method requires approximately $\log_2(N)$ steps to locate the closest point, making it efficient even for large datasets.
Bisection is particularly useful when interpolation requests are uncorrelated—meaning there is no pattern in the sequence of target points that could be exploited for faster searching.

In [ ]:
def bisection(xs, target):
    l, h = 0, len(xs) - 1
    while h - l > 1:
        m = (h + l) // 2
        if target >= xs[m]:
            l = m
        else:
            h = m
    return l # returns index of the closest value less than or equal to target

The above function efficiently narrows down the interval to locate the index of the nearest value.

We can perform some tests:

In [ ]:
for _ in range(10):
    xs = np.sort(np.random.uniform(0, 100, 10))
    v  = np.random.uniform(min(xs), max(xs))
    i  = bisection(xs, v)
    print(f'{xs[i]} <= {v} < {xs[i+1]}')

### Hunting Method

For cases where interpolation points are close together in sequence---common in applications with gradually changing target values---the hunting method offers faster performance than bisection from scratch.
Hunting takes advantage of the idea that, if the previous interpolation point is nearby, the search can start close to the last found position and "hunt" outward in expanding steps to bracket the target value.
Once the bracket is located, the search is refined using a quick bisection.

The hunting method is beneficial for correlated data requests, where successive target values are close, as it can skip large portions of the data and converge faster than starting from scratch each time.

In [ ]:
def hunt(xs, target, i_last):
    n = len(xs)
    assert 0 <= i_last < n - 1

    # Determine the search direction based on the target value
    if target >= xs[i_last]:
        l, h, step = i_last, min(n-1, i_last+1), 1
        while h < n - 1 and target > xs[h]:
            l, h = h, min(n-1, h+step)
            step *= 2
    else:
        l, h, step = max(0, i_last-1), i_last, 1
        while l > 0 and target < xs[l]:
            l, h = max(0, l-step), l
            step *= 2

    # Refine with bisection within the bracketed range
    return bisection(xs[l:h+1], target) + l

In [ ]:
i = 5
for _ in range(10):
    xs = np.sort(np.random.uniform(0, 100, 10))
    v  = np.random.uniform(min(xs), max(xs))
    i  = hunt(xs, v, i)
    print(f'{xs[i]} <= {v} < {xs[i+1]}')

### Linear Interpolation Using the Hunting Method

Once the nearest position is identified, interpolation proceeds with the closest data points.
Here, we implement a simple linear interpolation using the hunting method to locate the starting position, then use it to calculate the interpolated value.

In [ ]:
class Interpolator:
    def __init__(self, xs, ys):
        assert len(xs) == len(ys)
        self.xs, self.ys = xs, ys
        self.i_last = len(xs)//2

    def __call__(self, target, search_method='hunt'):
        if search_method == 'hunt':
            i = hunt(self.xs, target, self.i_last)
        elif search_method == 'bisection':
            i = bisection(self.xs, target)
        else:
            i = linear(self.xs, target)
        self.i_last = i  # Update last position for future hunts

        # Linear interpolation using the two nearest points
        x0, x1 = self.xs[i], self.xs[i + 1]
        y0, y1 = self.ys[i], self.ys[i + 1]

        return (y1 - y0) * (target - x0) / (x1 - x0) + y0

In [ ]:
def f(x):
    return np.exp(-0.5 * x * x)

Xs = np.sort(np.random.uniform(-5, 5, 20))
Ys = f(Xs)

fi = Interpolator(Xs, Ys)

In [ ]:
xs = np.linspace(min(Xs), max(Xs), 100)
ys = np.array([fi(x) for x in xs])

plt.plot(xs, ys, '.-', label='Interpolated points')
plt.plot(Xs, Ys, 'o',  label='Sampling data')
plt.legend()

Let's test if our claim in terms of performance works in real life.

In [ ]:
Xs = np.sort(np.random.uniform(-5, 5, 100))
Ys = f(Xs)
fi = Interpolator(Xs, Ys)

xs = np.linspace(min(Xs), max(Xs), 10000)

In [ ]:
%timeit ys = np.array([fi(x, search_method='linear') for x in xs])

In [ ]:
%timeit ys = np.array([fi(x, search_method='bisection') for x in xs])

In [ ]:
%timeit ys = np.array([fi(x, search_method='hunt') for x in xs])

## Polynomial Interpolation and Extrapolation

Given $M$ data points $(x_0, y_0), (x_1, y_1), \dots, (x_{M-1}, y_{M_1})$, there exists a unique polynomial of degree $M-1$ that pass through all $M$ points exactly.

### Lagrange's formula

This polynomial is given by Lagrange's classical formula,
\begin{align}
P_{M-1}(x)
&= \frac{(x-x_1)(x-x_2)\dots(x-x_{M-1})}{(x_0-x_1)(x_0-x_2)\dots(x_0-x_{M-1})} y_0 \\
&+ \frac{(x-x_0)(x-x_2)\dots(x-x_{M-1})}{(x_1-x_0)(x_1-x_2)\dots(x_1-x_{M-1})} y_1 + \dots \\
&+ \frac{(x-x_0)(x-x_2)\dots(x-x_{M-2})}{(x_{M-1}-x_0)(x_{M-1}-x_1)\dots(x_{M-1}-x_{M-2})} y_{M-1}
\end{align}
Using summation and product notations, one may rewrite Lagrange's formula as
\begin{align}
P_{M-1}(x)
= \sum_{m=0}^{M-1} \frac{\prod_{n=0,n\ne m}^{M-1}(x-x_n)}{\prod_{n=0,n\ne m}^{M-1}(x_m-x_n)} y_m
\end{align}
Substituting $x = x_{m'}$ for $0 \le m; < M$, it is straightforward to show
\begin{align}
P_{M-1}(x_{m'})
= \sum_{m=0}^{M-1} \delta_{mm'} y_m
\end{align}
and hence $P_{M-1}(x)$ does pass through all data points.